# Lecture 4: Stochastic Gradient Descent
***

<img src="figs/mountains.jpg" width=1100 height=50>

<br>

### Problem 1: Unregularized Stochastic Gradient Ascent on Text Data 
***

Suppose you have two messages, the first labeled positive (y = 1) and the second labeled negative (y = 0):

| POS Message | NEG Message  | 
|:-----------:|:------------:|
| AAAABBBC    |  BBCCCD      |

**Q**: Your first task is to encode the messages into feature vectors using the Bag-of-Words approach.  You may assume that "A", "B", "C", and "D" are the  only symbols in the vocabulary.  Don't forget to add a feature corresponding to the **bias** term!  

In [30]:
import numpy as np
import math
x1 = np.array([1, 4, 3, 1, 0])
y1 = 1
x2 = np.array([1, 0, 2, 3, 1])
y2 = 0
w0 = np.zeros(5)


**Q**: Your next task is to do one full pass through the data with **unregularized** SGA with the initial weight vector ${\bf w}$ set to zero and the learning rate set to $\eta = 1.0$.  You may assume that after the shuffle the order is POS then NEG.  

In [28]:
sigmoid(5)

0.9933071490757153

In [33]:
eta = 1

def sigmoid(x):
    return 1/(1+math.exp(-x))

def gradient(w, x, y, eta):
    a = sigmoid(np.dot(np.transpose(w),x))-y
    w2 = w - eta*a*x
    return w2

w1 = gradient(w0, x1, y1, eta)
w2 = gradient(w1, x2, y2, eta)
print('w1: {}'.format(w1))
print('w2: {}'.format(w2))

w1: [ 0.5  2.   1.5  0.5  0. ]
w2: [-0.49330715  2.         -0.4866143  -2.47992145 -0.99330715]


### Problem 2: A More Efficient Regularization 
***

Recall that we commonly include in the log-likelihood function a regularization term (on all weights except for the bias) of the form $\lambda \sum_{k=1}^D w_k^2$.   Incidentally, there are many forms of regularization.  This one is called L2-regularization because it can also be written as $\lambda\|{\bf w}_{1:D}\|_2^2$ where the norm here is the 2-norm or the *Euclidean* norm. For SGA this has the effect of including a term like $2\lambda w_k$ in the $k^\textrm{th}$ component of the gradient. 

**Q**: Why might this be terribly inefficient, especially in the context of text learning? 

**Answer**  
The reason this could be very inefficient is because regularization occurs for every parameter, ${\bf w}_k$, regardless of whether or not it is in the ${\bf w}_k^T$

A workaround for this is to use a *slightly* different form of L2 regularization, which we now describe.  

Consider the update step for the $k^\textrm{th}$ component of the weight vector in SGA with regularization.  It looks like

$$
w_k \leftarrow w_k - \eta\left([\textrm{sigm}({\bf w}^T{\bf x}_i)-y_i]x_{ik} + 2\lambda w_k \right)
 = (w_k - \eta[\textrm{sigm}({\bf w}^T{\bf x}_i)-y_i]x_{ik}) - 2\eta\lambda w_k 
$$

We're going to break the update up into two parts.  First we'll do the unregularized gradient update for $k$ corresponding only to **nonzero-features** in ${\bf x}_i$: 

$$
w_k \leftarrow w_k - \eta[\textrm{sigm}({\bf w}^T{\bf x}_i)-y_i]x_{ik}
$$

And then (in theory) we'll do the regularization update as follows 

$$
w_k \leftarrow w_k - 2\eta \lambda w_k = w_k (1 - 2\eta \lambda)  
$$

**Q**: Look carefully at the two update steps.  Why is this not the same as standard L2-regularization? 

**Answer:**  
In the above equation, the $w_k$ used in the second update step is the $w_k$ from the first update step, when, in reality, it should be the original, un-updated version of $w_k$.

OK, so far this doesn't look very helpful.  It seems like we still need to do the second update step for nonzero-weights, even if the $k^\textrm{th}$ feature is a zero-feature. It turns out that we can actually hold off on doing the regularization update for a weight until the time when we have to do a gradient update as well.  

The property of the update that makes this possible is that the regularization update is **multiplicative**.  This means that doing **three** updates of the form 

$$
\begin{array}{c}
~ w_k \leftarrow w_k (1 - 2\eta \lambda) \\
~ w_k \leftarrow w_k (1 - 2\eta \lambda) \\
~ w_k \leftarrow w_k (1 - 2\eta \lambda) \\
\end{array}
$$

is equivalent to 

$$
w_k \leftarrow w_k (1 - 2\eta \lambda)^3
$$

This means that we can hold off on doing the regularization update for the $k^\textrm{th}$ weight until we encounter a training example that has a $k^{\textrm{th}}$ nonzero feature.  The regularization update is then performed where the **shrinkage factor** is raised to the power of the number of iterations since the feature was last updated. 

The only mechanism we need to do this is a data structure to keep track of the last iteration that a particular component of the weight vector was updated.  Typically this is done with a hash table.  In your next homework assignment you'll use a dictionary. 

This processes is called **Lazy Sparse Regularization** and you can read more about it in a slightly different context <a href="https://lingpipe.files.wordpress.com/2008/04/lazysgdregression.pdf">here</a>. 

**Q**: Redo Problem 1 with Lazy Sparse Regularization with $\lambda = \frac{1}{4}$.

<br><br><br><br><br>
<br><br><br><br><br>
<br><br><br><br><br>
<br><br><br><br><br>
### Helper Functions
***

In [1]:
from IPython.core.display import HTML
HTML("""
<style>
.MathJax nobr>span.math>span{border-left-width:0 !important};
</style>
""")